## Day 47 Lecture 2 Assignment

In this assignment, we will perform K-Medoids clustering using responses to a survey about student life at a university.

In [2]:
!pip install pyclustering

     |████████████████████████████████| 2.7MB 5.7MB/s 
  Created wheel for pyclustering: filename=pyclustering-0.10.0.1-cp36-none-any.whl size=2615590 sha256=9d4efd736613d48de590f4790e8e4cb0a18c655b99852a9637fa7836cfad532e
  Stored in directory: /root/.cache/pip/wheels/7c/eb/8d/ce15daecf5a50483a8fbaf1d1e3cb2d7bbaeb0ad709af32419
Successfully built pyclustering


In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from pyclustering.cluster.kmedoids import kmedoids
import random

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [4]:
# answer goes here

u_df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv')

u_df.shape



(2958, 36)

For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [7]:
# answer goes here

X = u_df.filter(regex='Q5')
X.head(10)



,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0
3,1,1,0,1,1,1,1,0,1,0
4,0,1,1,0,1,0,0,0,1,0
5,0,1,0,0,1,0,0,0,0,0
6,0,1,1,0,0,0,0,1,0,0
7,1,0,0,0,0,0,0,1,0,0
8,1,1,0,0,1,1,0,0,1,0
9,1,1,1,0,0,0,0,1,1,0


The pyclustering implementation of kmedoids supports a variety of distance metrics, but they are primarily for numeric data. We will be using SMC/Hamming dissimilarity and precomputing the similarity matrix (an alternative would be to specify a user-defined function, which you are welcome to try in addition).

We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

In [9]:
# answer goes here

res = pdist(X, 'hamming')
res_pd = pd.DataFrame(squareform(res), index=X.index, columns=X.index)
res_pd.head(10)



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2918,2919,2920,2921,2922,2923,2924,2925,2926,2927,2928,2929,2930,2931,2932,2933,2934,2935,2936,2937,2938,2939,2940,2941,2942,2943,2944,2945,2946,2947,2948,2949,2950,2951,2952,2953,2954,2955,2956,2957
0,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
1,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2,0.1,0.1,0.0,0.7,0.4,0.2,0.1,0.2,0.5,0.3,0.2,0.2,0.4,0.5,0.3,0.2,0.2,0.3,0.3,0.4,0.2,0.3,0.2,0.1,0.2,0.3,0.2,0.1,0.4,0.0,0.2,0.2,0.4,0.2,0.4,0.0,0.2,0.2,0.4,0.1,...,0.3,0.1,0.1,0.0,0.1,0.3,0.0,0.1,0.2,0.1,0.3,0.1,0.2,0.3,0.2,0.1,0.1,0.0,0.1,0.0,0.2,0.2,0.1,0.3,0.1,0.2,0.3,0.0,0.3,0.1,0.2,0.4,0.3,0.4,0.4,0.3,0.1,0.1,0.1,0.1
3,0.6,0.6,0.7,0.0,0.5,0.5,0.8,0.7,0.2,0.6,0.7,0.5,0.5,0.4,0.6,0.7,0.7,0.6,0.6,0.7,0.7,0.6,0.7,0.6,0.5,0.6,0.7,0.8,0.5,0.7,0.7,0.7,0.7,0.5,0.5,0.7,0.7,0.7,0.7,0.6,...,0.6,0.6,0.8,0.7,0.6,0.4,0.7,0.6,0.7,0.6,0.4,0.6,0.5,0.6,0.5,0.8,0.6,0.7,0.6,0.7,0.5,0.5,0.6,0.6,0.6,0.7,0.6,0.7,0.4,0.6,0.7,0.5,0.4,0.5,0.3,0.6,0.6,0.6,0.6,0.8
4,0.3,0.3,0.4,0.5,0.0,0.2,0.3,0.6,0.3,0.3,0.4,0.4,0.4,0.5,0.5,0.6,0.4,0.5,0.3,0.4,0.2,0.3,0.4,0.3,0.4,0.3,0.2,0.3,0.4,0.4,0.4,0.2,0.4,0.4,0.4,0.4,0.2,0.4,0.4,0.3,...,0.3,0.3,0.3,0.4,0.3,0.3,0.4,0.5,0.2,0.3,0.3,0.3,0.2,0.3,0.4,0.5,0.5,0.4,0.3,0.4,0.4,0.4,0.3,0.5,0.3,0.2,0.3,0.4,0.3,0.5,0.2,0.2,0.3,0.4,0.2,0.3,0.3,0.3,0.5,0.3
5,0.1,0.1,0.2,0.5,0.2,0.0,0.3,0.4,0.3,0.5,0.4,0.4,0.4,0.5,0.3,0.4,0.4,0.3,0.3,0.4,0.4,0.3,0.4,0.1,0.4,0.3,0.4,0.3,0.4,0.2,0.4,0.4,0.4,0.2,0.4,0.2,0.2,0.4,0.4,0.1,...,0.3,0.3,0.3,0.2,0.1,0.3,0.2,0.3,0.2,0.1,0.3,0.1,0.2,0.1,0.2,0.3,0.3,0.2,0.1,0.2,0.4,0.2,0.1,0.5,0.1,0.2,0.3,0.2,0.1,0.3,0.2,0.2,0.3,0.4,0.2,0.3,0.1,0.1,0.3,0.3
6,0.2,0.2,0.1,0.8,0.3,0.3,0.0,0.3,0.6,0.2,0.1,0.3,0.3,0.4,0.4,0.3,0.1,0.4,0.2,0.3,0.1,0.2,0.1,0.2,0.3,0.2,0.1,0.0,0.3,0.1,0.1,0.1,0.3,0.3,0.5,0.1,0.1,0.1,0.3,0.2,...,0.2,0.2,0.0,0.1,0.2,0.4,0.1,0.2,0.1,0.2,0.4,0.2,0.3,0.4,0.3,0.2,0.2,0.1,0.2,0.1,0.3,0.3,0.2,0.2,0.2,0.1,0.2,0.1,0.4,0.2,0.1,0.3,0.4,0.3,0.5,0.2,0.2,0.2,0.2,0.0
7,0.3,0.3,0.2,0.7,0.6,0.4,0.3,0.0,0.5,0.3,0.4,0.2,0.4,0.5,0.1,0.0,0.4,0.1,0.5,0.4,0.4,0.5,0.4,0.3,0.4,0.5,0.4,0.3,0.6,0.2,0.4,0.4,0.4,0.2,0.2,0.2,0.4,0.4,0.4,0.3,...,0.5,0.3,0.3,0.2,0.3,0.3,0.2,0.3,0.4,0.3,0.3,0.3,0.4,0.3,0.4,0.1,0.3,0.2,0.3,0.2,0.2,0.4,0.3,0.3,0.3,0.4,0.5,0.2,0.5,0.3,0.4,0.6,0.3,0.4,0.4,0.5,0.3,0.3,0.3,0.3
8,0.4,0.4,0.5,0.2,0.3,0.3,0.6,0.5,0.0,0.4,0.7,0.3,0.3,0.6,0.4,0.5,0.5,0.4,0.4,0.7,0.5,0.6,0.7,0.4,0.5,0.4,0.5,0.6,0.5,0.5,0.7,0.5,0.5,0.3,0.3,0.5,0.5,0.7,0.7,0.4,...,0.6,0.4,0.6,0.5,0.4,0.2,0.5,0.6,0.5,0.4,0.2,0.4,0.3,0.4,0.5,0.6,0.4,0.5,0.4,0.5,0.3,0.3,0.4,0.6,0.4,0.5,0.6,0.5,0.4,0.6,0.5,0.3,0.2,0.5,0.1,0.4,0.4,0.4,0.6,0.6
9,0.4,0.4,0.3,0.6,0.3,0.5,0.2,0.3,0.4,0.0,0.3,0.1,0.3,0.4,0.4,0.3,0.3,0.4,0.4,0.5,0.1,0.4,0.3,0.4,0.3,0.4,0.1,0.2,0.5,0.3,0.3,0.1,0.3,0.3,0.3,0.3,0.3,0.3,0.5,0.4,...,0.4,0.2,0.2,0.3,0.4,0.2,0.3,0.4,0.3,0.4,0.2,0.4,0.3,0.6,0.5,0.4,0.4,0.3,0.4,0.3,0.1,0.5,0.4,0.2,0.4,0.3,0.4,0.3,0.6,0.4,0.3,0.5,0.2,0.3,0.3,0.4,0.4,0.4,0.4,0.2


Using the dissimilarity matrix, perform kmedoids clustering using k=2. Set the initial medoids randomly. Note that pyclustering expects the distance matrix to be a numpy array; a pandas dataframe may cause errors. 

Which survey responses are chosen as the cluster representatives? Print out the details of these responses.

In [10]:
# answer goes here

initial_medoids = X.sample(2).index

kmedoids_instance = kmedoids(res_pd, initial_medoids, data_type="distance_matrix")



In [11]:

kmedoids_instance.process()

clusters = kmedoids_instance.get_clusters()

medoids = kmedoids_instance.get_medoids()

In [12]:

X["cluster"] = 0

for i in range(len(clusters)):
    X.loc[clusters[i], "cluster"] = i

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


If you run the previous cell a few times, you'll probably notice that the medoids are very sensitive to initialization. A common approach to produce well-separated clusters is to choose initial medoids that are far apart. Re-run the previous process, except with a random pair of medoids that have a dissimilarity of 0.8 or higher. Are the results more stable now? How would you describe the typical clusters you see?

In [13]:
# answer goes here

hd = []
for row in range(res_pd.shape[0]):
    for col in range(res_pd.shape[1]):
        if res_pd._get_value(row,col) >= 0.8:
            hd.append([row, col])
            break

initial_medoids = random.sample(hd, 1)
initial_medoids[0]

[1996, 628]

In [14]:
initial_medoids = random.sample(hd, 1)
kmedoids_instance = kmedoids(res_pd, initial_medoids[0], data_type="distance_matrix")
kmedoids_instance.process()
clusters = kmedoids_instance.get_clusters()
medoids = kmedoids_instance.get_medoids()
X["cluster"] = 0

for i in range(len(clusters)):
    X.loc[clusters[i], "cluster"] = i

X.loc[medoids].style.background_gradient()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if _

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,cluster
2765,0,1,0,0,0,0,0,0,0,0,0
245,1,1,1,1,1,1,1,1,1,0,1
